In [1]:
import os, sys
import json
import requests
import nltk
from nltk.corpus import wordnet

In [2]:
datasets_path = '../datasets/'
maven_dataset_path = os.path.join(datasets_path, 'maven/')
maven_train_path = os.path.join(maven_dataset_path, "train.jsonl")
maven_valid_path = os.path.join(maven_dataset_path, "valid.jsonl")
maven_test_path = os.path.join(maven_dataset_path, "test.jsonl")

amuse_url = 'http://127.0.0.1:3002/api/model' # put here your endpoint for AMuSE-WSD!
def amuse_request(text = "Marco is eating an apple.", lang = "EN"):
    if type(text) == list:
        http_input = []
        for sentence in text:
            http_input.append({'text':sentence, 'lang':lang})
    else:
        http_input = [{'text':text, 'lang':lang}]
    res = requests.post(amuse_url, json = http_input)
    if res.status_code == 200:
        return json.loads(res.text)
    else:
        None

In [3]:
! wget -c https://cloud.tsinghua.edu.cn/f/e4beae178fe24753abf9/?dl=1 -O $maven_train_path
! wget -c https://cloud.tsinghua.edu.cn/f/f60818356a244aecb864/?dl=1 -O $maven_valid_path
! wget -c https://cloud.tsinghua.edu.cn/f/0893e8183a2a48ab85db/?dl=1 -O $maven_test_path

--2022-12-17 12:22:59--  https://cloud.tsinghua.edu.cn/f/e4beae178fe24753abf9/?dl=1
Resolving cloud.tsinghua.edu.cn (cloud.tsinghua.edu.cn)... 166.111.6.101, 2402:f000:1:406:166:111:6:101
Connecting to cloud.tsinghua.edu.cn (cloud.tsinghua.edu.cn)|166.111.6.101|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cloud.tsinghua.edu.cn/seafhttp/files/5a626b4d-6c59-4273-92e7-db9ce8a8b9be/train.jsonl [following]
--2022-12-17 12:23:01--  https://cloud.tsinghua.edu.cn/seafhttp/files/5a626b4d-6c59-4273-92e7-db9ce8a8b9be/train.jsonl
Reusing existing connection to cloud.tsinghua.edu.cn:443.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

--2022-12-17 12:23:01--  https://cloud.tsinghua.edu.cn/f/f60818356a244aecb864/?dl=1
Resolving cloud.tsinghua.edu.cn (cloud.tsinghua.edu.cn)... 166.111.6.101, 2402:f000:1:406:166:111:6:101
Connecting to cloud.tsinghua.edu.cn (cloud.tsinghua.ed

In [4]:
with open(maven_train_path, 'r') as json_file:
    maven_train = list([json.loads(json_line) for json_line in json_file])

In [5]:
maven_train[0]

{'title': '2006 Pangandaran earthquake and tsunami',
 'id': '8307a6b61b84d4eea42c1dd5e6e2cdba',
 'content': [{'sentence': 'The 2006 Pangandaran earthquake and tsunami occurred on July 17 at along a subduction zone off the coast of west and central Java, a large and densely populated island in the Indonesian archipelago.',
   'tokens': ['The',
    '2006',
    'Pangandaran',
    'earthquake',
    'and',
    'tsunami',
    'occurred',
    'on',
    'July',
    '17',
    'at',
    'along',
    'a',
    'subduction',
    'zone',
    'off',
    'the',
    'coast',
    'of',
    'west',
    'and',
    'central',
    'Java',
    ',',
    'a',
    'large',
    'and',
    'densely',
    'populated',
    'island',
    'in',
    'the',
    'Indonesian',
    'archipelago',
    '.']},
  {'sentence': 'The shock had a moment magnitude of 7.7 and a maximum perceived intensity of IV ("Light") in Jakarta, the capital and largest city of Indonesia.',
   'tokens': ['The',
    'shock',
    'had',
    'a',
 

In [6]:
# intermediate_dataset = []
# for el in maven_train:
#     events_phrases = {i:[] for i in range(0,len(el['content']))}
#     for e in el['events']:
#         for mention in e['mention']:
#             events_phrases[int(mention['sent_id'])].append({'trigger_word':mention['trigger_word'], 'offset':mention['offset']})

#     for i, sentence_tokens in enumerate(el['content']):
#         events = events_phrases[i]
#         intermediate_dataset.append({ 'sentence':sentence_tokens['sentence'], 'tokens':sentence_tokens['tokens'], 'events':events })
    
#     break
# intermediate_dataset

intermediate_dataset = [] # list of sentences
for el in maven_train:
    for i, sentence_tokens in enumerate(el['content']):
        intermediate_dataset.append(sentence_tokens['sentence'])
intermediate_dataset

['The 2006 Pangandaran earthquake and tsunami occurred on July 17 at along a subduction zone off the coast of west and central Java, a large and densely populated island in the Indonesian archipelago.',
 'The shock had a moment magnitude of 7.7 and a maximum perceived intensity of IV ("Light") in Jakarta, the capital and largest city of Indonesia.',
 "There were no direct effects of the earthquake's shaking due to its low intensity, and the large loss of life from the event was due to the resulting tsunami, which inundated a portion of the Java coast that had been unaffected by the earlier 2004 Indian Ocean earthquake and tsunami that was off the coast of Sumatra.",
 'The July 2006 earthquake was also centered in the Indian Ocean, from the coast of Java, and had a duration of more than three minutes.',
 'An abnormally slow rupture at the Sunda Trench and a tsunami that was unusually strong relative to the size of the earthquake were both factors that led to it being categorized as a ts

In [7]:
va_res_path = '../va_resources/'
os.makedirs(va_res_path, exist_ok=True)
va_folder_path = os.path.join(va_res_path,'VerbAtlas-1.1.0')
va_root_tsvs = os.path.join(va_folder_path, 'VerbAtlas-1.1.0')

with open(os.path.join(va_res_path, 'positive_synsets.json'), 'r') as outfile:
    positive_synsets = json.load(outfile)

In [14]:
amuse_request(["Marco is running a marathon.", "The eating of Marco is very loud."])

[{'tokens': [{'index': 0,
    'text': 'Marco',
    'pos': 'PROPN',
    'lemma': 'Marco',
    'bnSynsetId': 'O',
    'wnSynsetOffset': 'O',
    'nltkSynset': 'O'},
   {'index': 1,
    'text': 'is',
    'pos': 'VERB',
    'lemma': 'be',
    'bnSynsetId': 'bn:00083182v',
    'wnSynsetOffset': '2616386v',
    'nltkSynset': 'be.v.02'},
   {'index': 2,
    'text': 'running',
    'pos': 'VERB',
    'lemma': 'run',
    'bnSynsetId': 'bn:00093181v',
    'wnSynsetOffset': '1914947v',
    'nltkSynset': 'run.v.29'},
   {'index': 3,
    'text': 'a',
    'pos': 'DET',
    'lemma': 'a',
    'bnSynsetId': 'O',
    'wnSynsetOffset': 'O',
    'nltkSynset': 'O'},
   {'index': 4,
    'text': 'marathon',
    'pos': 'NOUN',
    'lemma': 'marathon',
    'bnSynsetId': 'bn:00053333n',
    'wnSynsetOffset': '7460546n',
    'nltkSynset': 'marathon.n.02'},
   {'index': 5,
    'text': '.',
    'pos': 'PUNCT',
    'lemma': '.',
    'bnSynsetId': 'O',
    'wnSynsetOffset': 'O',
    'nltkSynset': 'O'}]},
 {'tokens': 

In [47]:
from tqdm import tqdm
from collections import Counter

# load the very long list of random sentences:
# sentences_testing_dataset = ["Marco is running a marathon.", "The eating of Marco is very loud."] # this is for testing 
sentences_testing_dataset = intermediate_dataset[:1000]

# load the unambiguous nominal events to be used to build our dataset:
with open(os.path.join(va_res_path, 'candidates_unambiguous.json'), 'r') as json_file:
    candidates_unambiguous = json.load(json_file)

# explore and build the resulting dataset:
chunk_size = 16
sentences_len = len(sentences_testing_dataset)
nominal_event_count = Counter()
nominal_event_results = []
pbar = tqdm(range(0,sentences_len,chunk_size), disable=False)
nominal_found = 0
pbar.set_description(f"Total found: {nominal_found}")
for c in pbar:
    phrases_chunk = sentences_testing_dataset[c:c+chunk_size]
    res_sentences = amuse_request(phrases_chunk)
    for res_text in res_sentences:
        predictates = ["_"]*len(res_text['tokens'])
        syn_associtated_to_predicate = ["_"]*len(res_text['tokens'])
        for i, token in enumerate(res_text['tokens']):
            wn_synset_name = token['nltkSynset']
            if wn_synset_name != 'O':
                pos_type = wn_synset_name.split('.')[1]
                if pos_type == 'n' and wn_synset_name in candidates_unambiguous:
                    predictates[i] = candidates_unambiguous[wn_synset_name]['frames'][0]
                    nominal_found += 1
                    pbar.set_description(f"Total found: {nominal_found}")
                    nominal_event_count[wn_synset_name] += 1
        
        if any([p != "_" for p in predictates]):
            nominal_event_results.append({
                "lemmas":       [token['lemma'] for token in res_text['tokens']], 
                'pos_tags':     [token['pos'] for token in res_text['tokens']],
                'predictates':  predictates,
                'words':        [token['text'] for token in res_text['tokens']]
            })

Total found: 659: 100%|██████████| 63/63 [00:19<00:00,  3.22it/s]


In [48]:
nominal_event_count.most_common()

[('war.n.01', 38),
 ('attack.n.01', 23),
 ('bombing.n.01', 16),
 ('rebellion.n.02', 14),
 ('invasion.n.01', 14),
 ('radiation.n.01', 14),
 ('rain.n.01', 14),
 ('victory.n.01', 12),
 ('conflict.n.01', 11),
 ('attack.n.05', 10),
 ('fire.n.01', 10),
 ('explosion.n.01', 10),
 ('shooting.n.02', 10),
 ('flood.n.01', 10),
 ('crash.n.02', 9),
 ('action.n.01', 9),
 ('attempt.n.01', 8),
 ('show.n.01', 8),
 ('kill.n.02', 8),
 ('murder.n.01', 7),
 ('order.n.01', 7),
 ('match.n.02', 7),
 ('weather.n.01', 7),
 ('electricity.n.02', 7),
 ('earthquake.n.01', 6),
 ('operation.n.05', 6),
 ('takeoff.n.01', 6),
 ('trial.n.04', 6),
 ('violence.n.01', 6),
 ('demonstration.n.03', 6),
 ('election.n.01', 6),
 ('campaign.n.03', 5),
 ('federation.n.03', 5),
 ('collision.n.02', 4),
 ('killing.n.02', 4),
 ('power.n.02', 4),
 ('sinking.n.01', 4),
 ('concert.n.01', 4),
 ('tournament.n.01', 4),
 ('political_campaign.n.01', 4),
 ('combat.n.01', 4),
 ('vibration.n.01', 4),
 ('resistance.n.11', 4),
 ('assassination.n.02'

In [35]:
nominal_event_results

[]

In [22]:
nominal_event_phrases

{'tolerance.n.03': [],
 'allowance.n.06': [],
 'license.n.04': [],
 'authorization.n.04': [],
 'abidance.n.03': [],
 'blessing.n.01': [],
 'toleration.n.02': [],
 'cooperation.n.01': [],
 'obedience.n.01': [],
 'movement.n.11': [],
 'action.n.01': [],
 'airmail.n.02': [],
 'mail.n.02': [],
 'cartage.n.01': [],
 'delivery.n.01': [],
 'journey.n.01': [],
 'travel.n.01': [],
 'ferry.n.02': [],
 'importing.n.01': [],
 'lighterage.n.02': [],
 'portage.n.03': [],
 'porterage.n.02': [],
 'technology.n.01': [],
 'occupation.n.01': [],
 'departure.n.01': [],
 'smuggling.n.01': [],
 'happening.n.01': [],
 'transshipment.n.01': [],
 'weather.n.01': [],
 'absorption.n.01': [],
 'sorption.n.01': [],
 'consumption.n.01': [],
 'absorption.n.02': [],
 'adsorption.n.01': [],
 'assimilation.n.03': [],
 'imbibition.n.01': [],
 'chemisorption.n.01': [],
 'resorption.n.01': [],
 'approach.n.01': [],
 'work.n.01': [],
 'adoption.n.02': [],
 'proceeding.n.01': [],
 'co-option.n.01': [],
 'choice.n.02': [],
 

# OLD

In [ ]:
def dfs(wn_synset, positive_synsets = []):
    if wn_synset.name() in positive_synsets:
        return True
    for wn_synset_hypernym in wn_synset.hypernyms():
        if dfs(wn_synset_hypernym, positive_synsets):
            return True
    return False